In [1]:
import numpy as np
from PIL import Image
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
from skimage import io
import fs
import torchvision
from torchvision import transforms
import torch

from torch.utils.data import Dataset, DataLoader
import torchvision

import multiprocessing
#import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
cuda = torch.cuda.is_available()
import time
device = torch.device("cuda" if cuda else "cpu")

In [2]:
from apex import amp

In [3]:
'''import torch.nn.functional as F
from torch.autograd import Variable

def one_hot(index, classes):
    size = index.size() + (classes,)
    view = index.size() + (1,)

    mask = torch.Tensor(*size).fill_(0)
    index = index.view(*view)
    ones = 1.

    if isinstance(index, Variable):
        ones = Variable(torch.Tensor(index.size()).fill_(1))
        mask = Variable(mask, volatile=index.volatile)

    return mask.scatter_(1, index, ones)


class FocalLoss(nn.Module):

    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps

    def forward(self, input, target):
        y = one_hot(target, input.size(-1))
        logit = F.softmax(input, dim=-1)
        logit = logit.clamp(self.eps, 1. - self.eps)

        loss = -1 * y * torch.log(logit) # cross entropy
        loss = loss * (1 - logit) ** self.gamma # focal loss

        return loss.sum()
'''

'import torch.nn.functional as F\nfrom torch.autograd import Variable\n\ndef one_hot(index, classes):\n    size = index.size() + (classes,)\n    view = index.size() + (1,)\n\n    mask = torch.Tensor(*size).fill_(0)\n    index = index.view(*view)\n    ones = 1.\n\n    if isinstance(index, Variable):\n        ones = Variable(torch.Tensor(index.size()).fill_(1))\n        mask = Variable(mask, volatile=index.volatile)\n\n    return mask.scatter_(1, index, ones)\n\n\nclass FocalLoss(nn.Module):\n\n    def __init__(self, gamma=0, eps=1e-7):\n        super(FocalLoss, self).__init__()\n        self.gamma = gamma\n        self.eps = eps\n\n    def forward(self, input, target):\n        y = one_hot(target, input.size(-1))\n        logit = F.softmax(input, dim=-1)\n        logit = logit.clamp(self.eps, 1. - self.eps)\n\n        loss = -1 * y * torch.log(logit) # cross entropy\n        loss = loss * (1 - logit) ** self.gamma # focal loss\n\n        return loss.sum()\n'

In [4]:
'''#check all the testing images, if they all have 3 color channels
test_path='C:/Users/zhaoh/Downloads/test image/'
for f in fs.open_fs(test_path).walk.files(filter=["*.jpg"]):
    name=os.path.join(test_path+str(f))
    #print(name)
    a=io.imread(name)
    if a.shape[2] != 3:
        print(name)
'''

'#check all the testing images, if they all have 3 color channels\ntest_path=\'C:/Users/zhaoh/Downloads/test image/\'\nfor f in fs.open_fs(test_path).walk.files(filter=["*.jpg"]):\n    name=os.path.join(test_path+str(f))\n    #print(name)\n    a=io.imread(name)\n    if a.shape[2] != 3:\n        print(name)\n'

In [ ]:
#rename all the training pictures to avoid strange file name
'''
test_path = 'C:/Users/zhaoh/Downloads/test image/'
i=0
for f in fs.open_fs(test_path).walk.files(filter=["*.jpg"]):
    name=os.path.join(test_path+str(f))
    #print(name)
        #print(name[0:44])

    os.rename(name,name[0:45]+str(i)+'.jpg')
    i=i+1


'''


In [5]:
'''data_dir="C:/Users/zhaoh/Downloads/car images/"
i=0
for p in fs.open_fs(data_dir).walk.files(filter=["*.jpg"]):
    filename=data_dir+p
    #if '远景' in filename and not '远景X'in filename:
    #if ("奥迪A4" or "奥迪A3" or "奥迪A6" or "奥迪A8" or "奥迪A7") in filename:
    if '沃尔沃S' in filename or ('沃尔沃XC' in filename and '沃尔沃亚太' in filename) or '沃尔沃V40' in filename or '沃尔沃V60' in filename or '沃尔沃V90' in filename:
                index=80
                #self.data_ids.append(index)
                #self.data_filenames.append(filename)
                #a[index]=a[index]+1#沃尔沃
        #i=i+1
print(i)
'''

'data_dir="C:/Users/zhaoh/Downloads/car images/"\ni=0\nfor p in fs.open_fs(data_dir).walk.files(filter=["*.jpg"]):\n    filename=data_dir+p\n    #if \'远景\' in filename and not \'远景X\'in filename:\n    #if ("奥迪A4" or "奥迪A3" or "奥迪A6" or "奥迪A8" or "奥迪A7") in filename:\n    if \'沃尔沃S\' in filename or (\'沃尔沃XC\' in filename and \'沃尔沃亚太\' in filename) or \'沃尔沃V40\' in filename or \'沃尔沃V60\' in filename or \'沃尔沃V90\' in filename:\n                index=80\n                #self.data_ids.append(index)\n                #self.data_filenames.append(filename)\n                #a[index]=a[index]+1#沃尔沃\n        #i=i+1\nprint(i)\n'

In [6]:
#车型字典：品牌名称，文件名过滤关键字，训练集编号（如果默认训练集编号直接使用enumerate的index）
dictionary={'奥迪轿车':["奥迪A4L","奥迪A3","奥迪A6L",'奥迪A8',"奥迪A7",0 ],
            '奥迪SUV':["奥迪Q2","奥迪Q3","奥迪Q5" ,"奥迪Q7","奥迪Q8",1],
            #'奥迪跑车':["奥迪TT/","奥迪A5",2],#
            '奔驰轿车':['奔驰A级',"奔驰C级" ,"奔驰E级" , "奔驰S级",3],
            '奔驰SUV':["奔驰GL" ,"奔驰G级" , "奔驰M级" , "奔驰EQC",4],
            #'奔驰跑车':["奔驰SL" ,"奔驰CL" ,"AMG GT",5],
            '奔驰厢式车':['福建奔驰',6],
            '宝马轿车':["宝马1系" ,'宝马2系旅行车',"宝马3系" ,"宝马5系" ,"宝马7系",7],
            '宝马SUV':["宝马X1" ,"宝马X2" ,"宝马X3" ,"宝马X4", "宝马X5","宝马X6","宝马X7",8],
            #'宝马跑车':["宝马8系" ,"宝马6系" , "宝马i8" ,"宝马Z4",9],#
            '本田轿车':["飞度","凌派","锋范" ,"雅阁" ,"INSPIRE" ,"享域", "思铂睿" , "思域" , "哥瑞" , "竞瑞",10 ],
            '本田SUV':["歌诗图","缤智","皓影" ,"冠道" ,"本田CR-V", "本田UR-V" , "本田XR-V",11 ],
            '本田MPV':["艾力绅","奥德赛" ,"杰德" ,12],
            '别克轿车':["VELITE 5" , "君威" ,"君越" ,"林荫大道" , "威朗" , "英朗" , "阅朗" , "别克微蓝" ,"VERANO",13 ],
            '别克SUV':['昂科',14],
            '别克MPV':['别克GL6', '别克GL8',15],#别克mpv#########以上全部检查完毕###########
            '比亚迪轿车':["速锐" ,"比亚迪e" ,"秦" , "比亚迪F" ,"比亚迪G",16],#
            '比亚迪SUV':["元" ,"宋" , "唐" ,"比亚迪S",17],
            #'比亚迪MPV':['比亚迪M',0],
            '宝骏':['宝骏E','宝骏3','宝骏5','宝骏7',18],
            #'保时捷':["Panamera" , "保时捷911" ,"保时捷718" , "918 Spyder" , "Boxster" , "Carrera GT" , "Cayenne" , "Macan" , "Taycan" ,19],
            '标致轿车':["标致206" ,"标致207" , "标致301" , "标致307", "标致308" , "标致408","标致508",20],
            '标致SUV':["标致2008" , "标致3008" , "标致4008" , "标致5008",21],
            '北京轿车':["绅宝D"  ,"BEIJING-U7",22],
            '北京SUV':["北京BJ90","北京BJ20","北京BJ40","北京BJ80","绅宝X" , "智行","BEIJING-X3",23],
            '宝沃SUV':["宝沃BX",24],
            #'宾利':["慕尚" , "欧陆" ,"飞驰" ,"添越",25],
            '奔腾轿车':['奔腾B',26],
            '奔腾SUV':["奔腾X40" ,"奔腾X80" , "奔腾T",27],
            '北汽SUV':["战旗" ,  "BJ 212" , "北京BW007" ,"勇士" , "北汽幻速S" ,"北汽新能源EX",28],
            '北汽MPV':["北汽幻速H" , "北汽昌河M" , "北汽威望M" , "北汽瑞丽",29],
            '北汽轿车':["北汽新能源EC" , "北汽新能源EU" , "北斗星",30],
            '北汽皮卡':["越铃" , "锐铃" ,"福瑞达",31],
            '长安轿车':["奔奔" , "悦翔" , "逸动" , "锐程" , "睿驰",32],
            '长安SUV':["长安CS" , "长安新能源E-Pro" , "凌轩" ,"长安欧尚",33],
            '长安厢式车':["长安V" ,"长安之星" , "睿行",34],
            '长安皮卡':["神骐" , "长安凯程F70" , "长安星卡L1" , "跨越王" ,"新豹MINI",35],
            '长城皮卡':["长城风骏" , "炮",36],
            '大众轿车':["Polo" , "桑塔纳" , "朗逸" ,"帕萨特" , "辉昂" , "宝来" , "高尔夫" , "迈腾" , "速腾" , "捷达",37],
            '大众SUV':["C-TREK蔚领" , "探影" , "探歌" , "探岳" , "蔚揽" ,"途锐" , "途观" , "途铠" ,"途昂",38],
            '大众MPV':["迈特威" , "夏朗",39],
            '东风SUV':["风行T" , "风神AX" , "风光" , "东风风度MX6" , "锐骐多功能车" , "奥丁",40],
            '东风厢型车':["东风小康" ,"帅客" , "俊风" ,"御风",41],
            '东风轿车':["东风风神A60" , "东风风神E70" , "东风风神L60" , "奕炫" , "俊风E" , "富康ES500",42],
            '东风皮卡':["御风P16" , "锐骐皮卡" , "俊风",43],
            #'道奇SUV':["酷威",44],
            '东南轿车':["菱悦" ,"东南A5翼舞" ,"东南DX",45],
            '丰田轿车':["YARiS L" ,"雷凌" , "威驰" ,"卡罗拉",46],
            '丰田SUV':["丰田C-HR" , "汉兰达" ,"奕泽IZOA" ,"RAV4荣放" ,"普拉多" , "兰德酷路泽" ,"FJ 酷路泽",47],
            '丰田MPV':["埃尔法" ,"威尔法" , "普瑞维亚",48],
            #'丰田客车':["HIACE", "柯斯达",49],
            '福特轿车':["福克斯","福睿斯","金牛座",50],
            '福特SUV':["翼搏" , "锐际", "翼虎","领界" , "撼路者","探险者","Ranger",51],
            #'福特跑车':['Mustang',52],
            '福特MPV':["全顺" , "途睿欧",53],
            #'福田MPV':["伽途GT","风景V5","蒙派克E","风景G5","伽途i" , "萨瓦纳",54],
            #'福田厢式车':["风景G7","风景G9","图雅诺","蒙派克S",55],
            #'福田皮卡':["祥菱V" ,"拓陆者",56],
            #'法拉利跑车':["法拉利",57],
            '广汽传祺轿车':["传祺GA" , '广汽丰田iA5','祺智EV' ,'Aion S(埃安S)',58],
            '广汽传祺SUV':["传祺GS" ,'广汽ix4' ,'祺智PHEV' ,'传祺GE3','Aion LX(埃安LX)',59],
            '广汽传祺MPV':["传祺GM",60],
            '观致汽车':["观致3" , "观致5",61],
            #'GMC汽车':["YUKON" ,"SAVANA" , "SIERRA",62],
            '长城SUV':["哈弗H" ,"哈弗F",63],
            #'红旗':['红旗',64],
            '海马轿车':["海马爱尚EV","海马E3","海马M3","福美来",65],
            '海马SUV':["海马8S","海马S",66],
            #'金龙厢式车':["海格H",67],
            '吉利轿车':["金刚","缤瑞","帝豪/","帝豪GL","远景/","博瑞",'嘉际新能源',68],
            '吉利SUV':['嘉际','吉利icon','远景X','博越','缤越','帝豪GS',"远景X",69],
            '吉普SUV':['广汽菲克Jeep','大切诺基(进口)','自由人','自由光','自由侠','自由客','牧马人','指南者','指挥官经典',70],
            '捷达汽车':["捷达V",71],
            '江淮家用车':['江淮iEV','嘉悦A5/','瑞风S',72],
            '江淮商用车':['凌铃','星锐','江淮V7','帅铃T','瑞风M','瑞风',73],
            '金杯':['金杯F50','阁瑞斯','金杯大海狮','金杯海狮','金杯快运','小海狮X','海星T','金杯T30',74],
            #'捷豹':['捷豹X' ,'捷豹F', '捷豹I-PACE','捷豹E-PACE',75] ,
            '捷途':['捷途X',76],
            '凯迪拉克轿车':["凯迪拉克ATS-L",'凯迪拉克CT', '凯迪拉克XTS','凯迪拉克SLS',77],
            '凯迪拉克suv':["凯迪拉克XT" , '凯雷德','凯雷德ESCALADE',78],
            #‘克莱斯勒’:["克莱斯勒300C" ,'大捷龙',79],
            '雷克萨斯轿车':["雷克萨斯CT" , '雷克萨斯IS','雷克萨斯ES', '雷克萨斯LS','雷克萨斯LC', '雷克萨斯RC','雷克萨斯GS',80],
            '雷克萨斯suv':["雷克萨斯UX",'雷克萨斯NX','雷克萨斯RX','雷克萨斯LX',81],
            '路虎suv':['路虎',0],
            #'林肯':['林肯',0],
            '领克':['领克',0],
            '铃木':['昌河铃木','雨燕','天语 SX4',0],
            '雷诺':['东风雷诺','Espace',0],
            #'兰博基尼':['Urus','Hurac','Aventador','Gallardo',0],
            #'劳斯莱斯':['幻影', '古思特','库里南', '魅影','曜影',0],
            '猎豹':['猎豹CS',0],
            '马自达轿车':['马自达3','阿特兹','睿翼','马自达CX',0],
            '名爵':['名爵6', '名爵3', '名爵7', '锐行' , '名爵ZS','名爵HS',0],
            #'迷你':['MINI O','MINI CLUBMAN','MINI COUNTRYMAN',0],
            #'玛莎拉蒂':['Ghibli','总裁','Levante','GranTurismo','GranCabrio',0],
            '纳智捷suv':['大7 SUV',0],
            '奇瑞轿车':['艾瑞泽','旗云','风云',0],
            '奇瑞suv':['瑞虎',0],
            '起亚轿车':['KX CROSS','起亚K','焕驰','福瑞迪' ,'凯绅', '秀尔',0],
            '起亚suv':['起亚KX' ,'智跑',0],
            '启辰suv':['启辰D60', '启辰e30' ,'启辰T60', '启辰T70', '启辰T90', '启辰M50V',0],
            '日产轿车':['骊威','骐达TIIDA','轩逸' ,'阳光' ,'蓝鸟','天籁','西玛',0],
            '日产suv':['劲客','逍客','途达','奇骏','楼兰',0],
            '日产厢式车':['日产NV200','纳瓦拉','日产D22',0],
            '荣威':['荣威3','荣威i', '荣威RX','荣威MARVEL X',0],
            '斯柯达轿车':['晶锐','明锐','昕动','昕锐','速派','柯米克','柯珞克','柯迪亚克',0],
            '三菱':['蓝瑟' , '翼神','广汽三菱','帕杰罗(进口)',0],
            '上汽大通':['上汽MAXUS D', '上汽MAXUS T', '上汽MAXUS G',0],
            '斯巴鲁':['力狮','斯巴鲁XV','森林人','斯巴鲁BRZ','傲虎',0],
            #'Smart':['smart fortwo' ,'smart forfour',0],
            'SWM斯威汽车':['SWM斯威汽车',0],
            '特斯拉':['Model 3','Model S','Model X',0],
            '沃尔沃':['沃尔沃S','沃尔沃亚太/沃尔沃XC','沃尔沃V40','沃尔沃V60','沃尔沃V90',0],
            '五菱厢式车':['五菱宏光','五菱荣光V','五菱之光/','五菱之光V',0],
            '五菱皮卡':['五菱荣光小卡','五菱荣光新卡','五菱之光小卡',0],
            'WEY SUV':['WEY',0],
            #'五十铃':[]
            '蔚来汽车':['蔚来ES',0],
            '现代轿车':['瑞纳','悦纳','北京现代i30','悦动','菲斯塔','领动','伊兰特EV','逸行','名图','索纳塔九','索纳塔插电混动',0],
            '现代suv':['昂希诺','北京现代ix','途胜','胜达',0],
            '雪佛兰轿车':['迈锐宝','科鲁','科沃兹','迈锐宝','赛欧','科迈罗',0],
            '雪佛兰suv':['科帕奇','创酷','创界','探界者',0],
            '雪铁龙':['雪铁龙C','爱丽舍','C4世嘉','云逸 C4 AIRCROSS','天逸 C5 AIRCROSS','C4 PICASSO',0],
            '新宝骏':['新宝骏RC-6','新宝骏RS-3','新宝骏RS-5','新宝骏RM-5',0],
            '英菲尼迪':['英菲尼迪Q50','英菲尼迪QX50','英菲尼迪QX80','英菲尼迪FX',0],
            #'依维柯':['依维柯',0],
            '天津一汽':['威志V5','夏利N','骏派',0],
            '众泰':['众泰Z','众泰T','众泰SR','大迈X','众泰E200',0],
            '中华':['中华H','中华V','中华骏捷F','中华骏捷',0]
           }



In [7]:
'''for index, item in enumerate(dictionary.items()):
    print(index,item)
'''

'for index, item in enumerate(dictionary.items()):\n    print(index,item)\n'

In [8]:
from torch.utils.data import Dataset, DataLoader
class DrivingDataset(Dataset):
    def __init__(self,data_dir, input_w=224, input_h=224,is_train=True,transform=None):
        """{0:"other",1:"drink",2:"phone",3:"smoke"}"""
        self.data_filenames = []
        self.data_ids = []
        self.is_train=is_train

        self.data_root=fs.open_fs(data_dir)
        self.transform = transform
        index=0
        
        countlist = [[0,''] for i in range(120)]
        #count the number in each category
        for p in self.data_root.walk.files(filter=["*.jpg"]):
            filename=data_dir+p
#########################################a            
            for index,item in enumerate(dictionary.items()):
                #print(index.dtype)
                for i in range (len(item[1])-1):
                    if item[1][i] in filename:#i每个车型
                        #print(countlist[index])
                        countlist[index][0]+=1
                        self.data_ids.append(index)
                        self.data_filenames.append(filename)
                        
             
            
            
  
        # print(self.data_filenames)
        #print(self.data_ids)
        
        print('total number of training images',len(self.data_ids))
        for index,item in enumerate(dictionary.items()):
            countlist[index][1]=item[0]
        print(countlist)
                
        

    def __getitem__(self, item):
        """Grey(i, j) = 0.299 × R(i, j) + 0.587 × G(i, j) + 0.114 × B(i, j)"""

        img_path = self.data_filenames[item]
        #print(img_path)
        target = self.data_ids[item]

        image = io.imread(img_path)#发现cv2不支持路径中有中文，换成io.imread
        if self.transform:
            image = self.transform(image)

        #resize_image = self.resize_padding(image, self.input_w, self.input_h)
        #transpose_image = resize_image[:, :, ::-1].copy()
        
        #img = self.transform(transpose_image)
        #print(img.shape)
        target = np.array([target], dtype=np.long)#########################
        target = torch.from_numpy(target)
        return image,target
    
    def __len__(self):
        return len(self.data_filenames)
    
    def resize_padding(self, x, inputwidth, inputheight):
        scale = float(float(inputwidth) / float(inputheight))
        h, w, c = x.shape
        if float(float(w) / float(h)) >= scale:
            # padding h
            dif = np.abs(w / scale - h)
            pad1, pad2 = int(dif // 2), int(dif - dif // 2)
            pad = ((pad1, pad2), (0, 0), (0, 0))
            input_x = np.pad(x, pad, "constant", constant_values=0)
            input_img = cv2.resize(input_x, (inputwidth, inputheight))
        else:
            # padding w
            dif = np.abs(h * scale - w)
            pad1, pad2 = int(dif // 2), int(dif - dif // 2)
            pad = ((0, 0), (pad1, pad2), (0, 0))
            input_x = np.pad(x, pad, "constant", constant_values=0)
            input_img = cv2.resize(input_x, (inputwidth, inputheight))

        return input_img

In [9]:
#pad=np.random.randint(25,size=1).item()
transform = transforms.Compose([
            transforms.ToPILImage(),
            #transforms.Pad(np.random.randint(25,size=1).item(), fill=255, padding_mode='constant'),
            transforms.Resize((224,224), interpolation=2),
            #transforms.RandomRotation(15),
            transforms.RandomResizedCrop((224,224), scale=(0.7, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=2),    
            transforms.RandomHorizontalFlip(),   
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
]) 

In [ ]:
'''
#count the number of training images##################################only for test

#test_path = 'C:/Users/zhaoh/Downloads/car recognition/data/honda/sedan/'
test_path='C:/Users/zhaoh/Downloads/car images/'
i=0
#carlabel=[[0 for x in range(10000)] for x in range(2)] 
#print(carlabel)
data_root=fs.open_fs(test_path)
for p in data_root.walk.files(filter=["*.jpg"]):
    i=i+1

print(i)
##############################################
'''

In [10]:
train_set = DrivingDataset(data_dir="C:/Users/zhaoh/Downloads/car images/", is_train=True,transform=transform) 
val_set = DrivingDataset(data_dir="C:/Users/zhaoh/Downloads/test image/", is_train=True,transform=transform) 
test_set = DrivingDataset(data_dir="C:/Users/zhaoh/Downloads/test image/", is_train=True,transform=transform) 
#756 1133 102 868 1367 790

total number of training images 58680
[[596, '奥迪轿车'], [1181, '奥迪SUV'], [1024, '奔驰轿车'], [1456, '奔驰SUV'], [180, '奔驰厢式车'], [1244, '宝马轿车'], [886, '宝马SUV'], [1491, '本田轿车'], [936, '本田SUV'], [202, '本田MPV'], [519, '别克轿车'], [259, '别克SUV'], [120, '别克MPV'], [1232, '比亚迪轿车'], [1693, '比亚迪SUV'], [1036, '宝骏'], [855, '标致轿车'], [389, '标致SUV'], [206, '北京轿车'], [509, '北京SUV'], [61, '宝沃SUV'], [34, '奔腾轿车'], [240, '奔腾SUV'], [454, '北汽SUV'], [245, '北汽MPV'], [64, '北汽轿车'], [34, '北汽皮卡'], [1077, '长安轿车'], [1570, '长安SUV'], [291, '长安厢式车'], [310, '长安皮卡'], [91, '长城皮卡'], [1757, '大众轿车'], [826, '大众SUV'], [58, '大众MPV'], [700, '东风SUV'], [633, '东风厢型车'], [136, '东风轿车'], [101, '东风皮卡'], [364, '东南轿车'], [857, '丰田轿车'], [852, '丰田SUV'], [101, '丰田MPV'], [299, '福特轿车'], [320, '福特SUV'], [142, '福特MPV'], [787, '广汽传祺轿车'], [601, '广汽传祺SUV'], [162, '广汽传祺MPV'], [74, '观致汽车'], [1477, '长城SUV'], [163, '海马轿车'], [127, '海马SUV'], [913, '吉利轿车'], [632, '吉利SUV'], [1396, '吉普SUV'], [0, '捷达汽车'], [476, '江淮家用车'], [907, '江淮商用车'], [81, '金杯'], [473, '捷途'], [431, '凯

In [11]:
batch_size = 12
#n_workers = multiprocessing.cpu_count()
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=True, num_workers=0)
valloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

In [12]:
'''i=0
for index, [image_batch, label] in enumerate(trainloader):#next(iter(trainloader))
        #print(image_batch[index].shape)
        #print(label.shape,index,image_batch.shape)
        
        for j in range(batch_size):
        #print(image_batch[i])
            if label[j]==0:
                img = np.moveaxis((image_batch[j].numpy()+2)/4,0,2)
                i+=1
                #print(i)
                plt.subplot(2,4,(i%8+1))
                plt.imshow(img)
                if i%8==0:
                    plt.show()
            
'''


'i=0\nfor index, [image_batch, label] in enumerate(trainloader):#next(iter(trainloader))\n        #print(image_batch[index].shape)\n        #print(label.shape,index,image_batch.shape)\n        \n        for j in range(batch_size):\n        #print(image_batch[i])\n            if label[j]==0:\n                img = np.moveaxis((image_batch[j].numpy()+2)/4,0,2)\n                i+=1\n                #print(i)\n                plt.subplot(2,4,(i%8+1))\n                plt.imshow(img)\n                if i%8==0:\n                    plt.show()\n            \n'

In [13]:
#define the network below,resnet 152
####################################
#######################################
###########################################
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url 


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
           'wide_resnet50_2', 'wide_resnet101_2']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,#########################################################
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    # Allow for accessing forward method in a inherited class
    forward = _forward


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model





def resnet152(pretrained=True, progress=True, **kwargs):
    r"""ResNet-152 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet152', Bottleneck, [3, 8, 36, 3], pretrained, progress,
                   **kwargs)





In [18]:
import torch.optim as optim
model = resnet152(pretrained=True)#torch.load('model2.pkl')

fc_features = model.fc.in_features

model.fc = nn.Linear(fc_features, 120)

#criterion = FocalLoss()
criterion_class= nn.CrossEntropyLoss() 
criterion_hinge=nn.MultiMarginLoss()

learning_rate = 1e-4
print(model)
model=model.to(device)
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

model, optimizer = amp.initialize(model, optimizer, opt_level="O1") # 这里是“欧一”，不是“零一”



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

)
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [19]:
from tqdm import tqdm
def train_epoch(model, train_loader, criterion1,criterion2, optimizer):
    model.train()
    model.to(device)

    running_loss = 0.0
    total_predictions = 0.0
    correct_predictions = 0.0
    
    start_time = time.time()
    for data, target in tqdm(train_loader):  
        #print(target)
        optimizer.zero_grad()   
        data = data.to(device)
        target = target.long().to(device)
        target=target.squeeze(1)#############
        outputs = model(data)
        loss = criterion1(outputs, target)####################
        loss+=criterion2(outputs, target)
        print(loss)
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += target.size(0)
        correct_predictions += (predicted == target).sum().item()
        acc = (correct_predictions/total_predictions)*100.0
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
            #loss.backward()
            optimizer.step()

    end_time = time.time()
    
    running_loss /= len(train_loader)
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time,' train accuracy',acc)
    return running_loss,acc

In [20]:
def test_model(model, test_loader, criterion1,criterion2):
    with torch.no_grad():
        model.eval()
        model.to(device)

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0
        Predicted=[]
        for data, target in test_loader: 
            #print(data.shape,target.shape)
            data = data.to(device) 
            target = target.long().to(device)
            target=target.squeeze(1)##################
            outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            Predicted.append(predicted)
            print('target',target,'prediction',predicted)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()

            loss = criterion1(outputs, target)####################
            loss+=criterion2(outputs, target)
            running_loss += loss.item()


        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        print('Testing Loss: ', running_loss)
        print('Testing Accuracy: ', acc, '%')
        
        return running_loss, acc
    

In [ ]:
model.load_state_dict(torch.load('parameter.pkl'))

In [21]:

n_epochs = 5
Train_loss = []
Test_loss = []
Test_acc = []
Val_loss = []
Val_acc = []
Train_acc=[]
for i in range(n_epochs):
    train_loss,train_acc = train_epoch(model, trainloader, criterion_class, criterion_hinge,optimizer)
    test_loss, test_acc = test_model(model, testloader, criterion_class, criterion_hinge)
    #val_loss, val_acc = test_model(model, valloader, criterion)
    Train_acc.append(train_acc)
    Train_loss.append(train_loss)
    #Val_acc.append(val_acc)
    #Val_loss.append(val_loss)
    Test_loss.append(test_loss)
    Test_acc.append(test_acc)
    print(i,'epoch='*20)
#test_loss, test_acc = test_model(model, testloader, criterion)


  0%|                                                                                         | 0/4890 [00:00<?, ?it/s]

tensor(5.9155, device='cuda:0', grad_fn=<AddBackward0>)


  0%|                                                                               | 1/4890 [00:00<1:17:39,  1.05it/s]

tensor(5.8567, device='cuda:0', grad_fn=<AddBackward0>)


  0%|                                                                               | 2/4890 [00:01<1:14:03,  1.10it/s]

...

  1%|▉                                                                             | 60/4890 [00:50<1:07:07,  1.20it/s]

tensor(4.8150, device='cuda:0', grad_fn=<AddBackward0>)


  1%|▉                                                                             | 61/4890 [00:51<1:05:52,  1.22it/s]

tensor(4.3170, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'parameter.pkl')

In [ ]:
test_loss, test_acc = test_model(model, testloader, criterion)


In [ ]:
test_path='C:/Users/zhaoh/Downloads/test image/'
for f in fs.open_fs(test_path).walk.files(filter=["*.jpg"]):
    name=os.path.join(test_path+str(f))
    print(name)
        #test_path =name
        #newname=os.path.join(test_path,'benz suv'+str(i)+'.jpg')
        #os.rename(name, newname)
        #test_image=newname
    filename = name
    
    img=io.imread(filename)
    
    img=cv2.resize(img,(224,224))
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    
    plt.imshow(img)
    outputs = model(np.reshape(torch.FloatTensor(img),(1,3,224,224)).to(device))
    _, predicted = torch.max(outputs.data, 1)
    print(outputs)
    plt.show()